In [1]:
require 'nn'
require 'hdf5'

local Squeeze, parent = torch.class('nn.Squeeze', 'nn.Module')

function Squeeze:updateOutput(input)
    self.size = input:size()
    self.output = input:squeeze()
    return self.output
end

function Squeeze:updateGradInput(input, gradOutput)
  self.gradInput = gradOutput:view(self.size)
  return self.gradInput  
end

myFile = hdf5.open('PTB.hdf5','r')
data = myFile:all()
myFile:close()

In [ ]:
train_input_word_windows = data['train_input_word_windows']
train_input_cap_windows = data['train_input_cap_windows']

train = train_input_word_windows:clone()
train_cap = train_input_cap_windows:clone()
train_output = data['train_output']

In [ ]:
dim_hidden = 50
dim_hidden2 = 50
--Define the module
neuralnet = nn.Sequential()

--Include the lookup tables
neuralnet:add(nn.LookupTable(data['nwords'][1],dim_hidden))
neuralnet:add(nn.View(1,-1,5*50))
neuralnet:add(nn.Squeeze()) -- this layer is to go from a 1xAxB tensor to AxB dimensional tensor (https://groups.google.com/forum/#!topic/torch7/u4OEc0GB74k)
neuralnet:add(nn.Linear(5*dim_hidden,dim_hidden2))
neuralnet:add(nn.HardTanh())
neuralnet:add(nn.Linear(dim_hidden2, data['nclasses'][1]))
neuralnet:add(nn.LogSoftMax())

In [ ]:
dataset={};
for i=1,train:size(1) do 
  dataset[i] = {train[i]:view(1,5), train_output[i]}
end
function dataset:size() return train:size(1) end -- 100 examples

In [ ]:
criterion = nn.ClassNLLCriterion()

In [ ]:
timer = torch.Timer()

trainer = nn.StochasticGradient(neuralnet, criterion)
trainer.learningRate = 0.001
trainer.maxIteration = 10
trainer:train(dataset)

print('Time elapsed: ' .. timer:time().real .. ' seconds',"\n")


In [ ]:

val_word = data['valid_input_word_windows']:clone()
val_output = data['valid_output']:clone()

pred_train = neuralnet:forward(train)
max,argmax_train = pred_train:max(2)
acc_train = 0
for i = 1, train_output:size(1) do
    if argmax_train[i][1] == train_output[i] then
        acc_train = acc_train + 1
    end
end
score_train = acc_train/train_output:size(1)
print('Validation Score on Train is '..score_train)

In [ ]:
pred_val = neuralnet:forward(val_word)
max,argmax_val = pred_val:max(2)
acc_val = 0
for i = 1, data['valid_output']:size(1) do
    if argmax_val[i][1] == data['valid_output'][i] then
        acc_val = acc_val + 1
    end
end
score_val = acc_val/data['valid_output']:size(1)
print('Validation Score on Validation is '..score_val)

## Adding the cap:

In [ ]:
train_new = train_input_word_windows:clone()
train_cap_new = train_cap:clone()
train_cap_new:add(100002)

In [ ]:
train_new = torch.cat(train_new,train_cap_new,2)

In [ ]:
dim_hidden = 50
dim_hidden2 = 50
--Define the module
neuralnet_wc = nn.Sequential()

--Include the lookup tables
neuralnet_wc:add(nn.LookupTable(data['nwords'][1]+4,dim_hidden))
neuralnet_wc:add(nn.View(1,-1,10*dim_hidden))
neuralnet_wc:add(nn.Squeeze()) -- this layer is to go from a 1xAxB tensor to AxB dimensional tensor (https://groups.google.com/forum/#!topic/torch7/u4OEc0GB74k)
neuralnet_wc:add(nn.Linear(10*dim_hidden,dim_hidden2))
neuralnet_wc:add(nn.HardTanh())
neuralnet_wc:add(nn.Linear(dim_hidden2, data['nclasses'][1]))
neuralnet_wc:add(nn.LogSoftMax())

In [ ]:
dataset2={};
for i=1,train:size(1) do 
  dataset2[i] = {train_new[i]:view(1,10), train_output[i]}
end
function dataset2:size() return train:size(1) end -- 100 examples

In [ ]:
trainer2 = nn.StochasticGradient(neuralnet_wc, criterion)
trainer2.learningRate = 0.01
trainer2.maxIteration = 3
trainer2:train(dataset2)

In [ ]:
pred_train2 = neuralnet_wc:forward(train_new)
max,argmax_train2 = pred_train2:max(2)
acc_train2 = 0
for i = 1, train_output:size(1) do
    if argmax_train2[i][1] == train_output[i] then
        acc_train2 = acc_train2 + 1
    end
end
score_train2 = acc_train2/train_output:size(1)
print('Validation Score on Train is '..score_train2)


In [ ]:
valid_new = data['valid_input_word_windows']:clone()
valid_cap_new = data['valid_input_cap_windows']:clone()
valid_cap_new:add(100002)
valid_new = torch.cat(valid_new,valid_cap_new,2)

In [ ]:
pred_val2 = neuralnet_wc:forward(valid_new)
max,argmax_val2 = pred_val2:max(2)
acc_val2 = 0
for i = 1, data['valid_output']:size(1) do
    if argmax_val2[i][1] == data['valid_output'][i] then
        acc_val2 = acc_val2 + 1
    end
end
score_val2 = acc_val2/data['valid_output']:size(1)
print('Validation Score on Train is '..score_val2)

# Adding the embeddings

In [ ]:
train_new = data['train_input_word_windows']
train_cap_new = data['train_input_cap_windows']
train_output = data['train_output']
train_cap_new:add(100002)
train_new = torch.cat(train_new,train_cap_new,2)

In [ ]:
dim_hidden = 50
dim_hidden2 = 50
--Define the module
neuralnet_wc = nn.Sequential()

par = nn.LookupTable(data['nwords'][1]+4,dim_hidden)
word_embeddings = data['word_embeddings']
par.weight:narrow(1, 1, data['nwords'][1]):copy(word_embeddings:narrow(1, 1, data['nwords'][1]))
print(par.weight[1][1], word_embeddings[1][1])

--Include the lookup tables
neuralnet_wc:add(par)
neuralnet_wc:add(nn.View(1,-1,10*dim_hidden))
neuralnet_wc:add(nn.Squeeze()) -- this layer is to go from a 1xAxB tensor to AxB dimensional tensor (https://groups.google.com/forum/#!topic/torch7/u4OEc0GB74k)
neuralnet_wc:add(nn.Linear(10*dim_hidden,dim_hidden2))
neuralnet_wc:add(nn.HardTanh())
neuralnet_wc:add(nn.Linear(dim_hidden2, data['nclasses'][1]))
neuralnet_wc:add(nn.LogSoftMax())

criterion = nn.ClassNLLCriterion()

In [ ]:
dataset2={};
for i=1,train_new:size(1) do 
  dataset2[i] = {train_new[i]:view(1,10), train_output[i]}
end
function dataset2:size() return train_new:size(1) end -- 100 examples

In [ ]:
dataset2[1][1]

In [ ]:
timer = torch.Timer()

trainer2 = nn.StochasticGradient(neuralnet_wc, criterion)
trainer2.learningRate = 0.01
trainer2.maxIteration = 3
trainer2:train(dataset2)

print('Time elapsed: ' .. timer:time().real .. ' seconds',"\n")


In [ ]:
-- Evaluation
valid = data['valid_input_word_windows']
valid_cap = data['valid_input_cap_windows']
valid_new = torch.cat(valid,torch.add(valid_cap,100002) ,2)

In [ ]:
-- On train set

pred_train2 = neuralnet_wc:forward(train_new)
max,argmax_train2 = pred_train2:max(2)
acc_train2 = 0
for i = 1, train_output:size(1) do
    if argmax_train2[i][1] == train_output[i] then
        acc_train2 = acc_train2 + 1
    end
end
score_train2 = acc_train2/train_output:size(1)
print('Validation Score on Train is '..score_train2)


-- On validation set
pred_val2 = neuralnet_wc:forward(valid_new)
max,argmax_val2 = pred_val2:max(2)
acc_val2 = 0
for i = 1, data['valid_output']:size(1) do
    if argmax_val2[i][1] == data['valid_output'][i] then
        acc_val2 = acc_val2 + 1
    end
end
score_val2 = acc_val2/data['valid_output']:size(1)
print('Validation Score on Train is '..score_val2)

# Hyperparameter tunning script

Run all the following cells until the one called SCRIPT TO TUNE. This one is the longest to run, there are 2 loops: one over the parameters to test (here over dim_hidden) and one over the number of iterations (5 iterations of 4 epoch), the latter is to log more information about loss and accuracy and attest that the model converges.
In between each iteration of the outer loop the script rewrites in the log files in case of failure.

Then the last cell is to load the saved files. The goal is to use the two log of accuracy_train and accuracy_valid to select the best model and then retrieve its test prediction to submit them and take the lead!!
We will have enough log to have some nice plots too.

In [2]:
-- Helper

function compute_accuracy(pred, true_)
    max,argmax = pred:max(2)
    acc = 0
    for i = 1, true_:size(1) do
        if argmax[i][1] == true_[i] then
            acc = acc + 1
        end
    end
    score = acc/true_:size(1)
    
    return score
end

-- If words_embeddings is nil, weight are initialized randomly by torch
function build_nn(nwords, ncap, nclasses, dim_hidden, dim_hidden2, word_embeddings)
    --Define the module
    neuralnet_wc = nn.Sequential()

    par = nn.LookupTable(nwords + ncap,dim_hidden)
    
    -- Adding the embeddings
    if word_embeddings then
        par.weight:narrow(1, 1, nwords):copy(word_embeddings:narrow(1, 1, nwords))
    end
    neuralnet_wc:add(par)

    neuralnet_wc:add(nn.View(1,-1,10*dim_hidden))
    neuralnet_wc:add(nn.Squeeze()) -- this layer is to go from a 1xAxB tensor to AxB dimensional tensor (https://groups.google.com/forum/#!topic/torch7/u4OEc0GB74k)
    neuralnet_wc:add(nn.Linear(10*dim_hidden,dim_hidden2))
    neuralnet_wc:add(nn.HardTanh())
    neuralnet_wc:add(nn.Linear(dim_hidden2, nclasses))
    neuralnet_wc:add(nn.LogSoftMax())
    
    return neuralnet_wc
end

In [3]:
-- Loading the data

-- dimension
nwords = data['nwords'][1]
ncap = 4
nclasses = data['nclasses'][1]

-- Training data
train_output = data['train_output']
train_new = torch.cat(data['train_input_word_windows'],
    torch.add(data['train_input_cap_windows'], 100002),2)

-- Validation data
valid_output = data['valid_output']
valid_new = torch.cat(data['valid_input_word_windows'],
    torch.add(data['valid_input_cap_windows'],100002) ,2)

-- Test data
test_new = torch.cat(data['test_input_word_windows'],
    torch.add(data['test_input_cap_windows'],100002) ,2)

-- Formating the dataset to train
dataset={};
for i=1,train_new:size(1) do 
  dataset[i] = {train_new[i]:view(1,10), train_output[i]}
end
function dataset:size() return train_new:size(1) end -- 100 examples

In [ ]:
-- Hyperparameter grid choice

-- Place to change parameter if needed
dim_hidden2_list = torch.DoubleTensor({40, 50, 60, 80, 100, 120})
num_parameters = dim_hidden2_list:size(1)

In [ ]:
-- Metric storage

-- Initialization
loss_train = torch.zeros(num_parameters, 5) -- Store every 4 epochs
training_accuracy = torch.zeros(num_parameters, 5) -- Store every 4 epochs
valid_accuracy = torch.zeros(num_parameters, 5) -- Store every 4 epochs
training_time = torch.zeros(num_parameters)
test_pred = torch.zeros(test_new:size(1),45)


In [ ]:
dim_hidden2_list:size(1)

In [ ]:
-- Script

-- REquired value because of the embeddings
dim_hidden = 50

for d = 1,dim_hidden2_list:size(1) do
--for d=1,1 do
    dim_hidden2 = dim_hidden2_list[d]

    --Define the model
    neuralnet_wc = build_nn(nwords, ncap, nclasses, dim_hidden, dim_hidden2, data['word_embeddings'])
    criterion = nn.ClassNLLCriterion()
    
    -- Training
    timer = torch.Timer()
    trainer = nn.StochasticGradient(neuralnet_wc, criterion)
    trainer.learningRate = 0.01
    trainer.maxIteration = 4
    
    for j=1, 5 do
        trainer:train(dataset)
        
        -- Pred on train
        pred_val = neuralnet_wc:forward(valid_new)
        valid_accuracy[d][j] = compute_accuracy(pred_val, valid_output)
        
        pred_train = neuralnet_wc:forward(train_new)
        training_accuracy[d][j] = compute_accuracy(pred_train, train_output)
        loss_train[d][j] = criterion:forward(pred_train, train_output)
    end

    training_time[d] = timer:time().real
    test_pred = neuralnet_wc:forward(test_new)
    
    -- Saving (rewriting the files with update at each iteration)
    filename = 'log_hyper_embedding.f5'
    myFile = hdf5.open(filename, 'w')
    myFile:write('valid_accuracy', valid_accuracy)
    myFile:write('training_accuracy', training_accuracy)
    myFile:write('training_time', training_time)
    myFile:write('loss_train', loss_train)
    myFile:close()
    
    -- Saving the pred for each hyperparameter
    filename = 'test_pred_embedding_h_'.. d .. '.f5'
    myFile = hdf5.open(filename, 'w')
    myFile:write('test_pred', test_pred)
    myFile:close()    
    
end

In [ ]:
--- Script to read the saved files

myFile = hdf5.open('log_hyper_embedding.f5','r')
hyperparameter_log = myFile:all()
myFile:close()

In [ ]:
hyperparameter_log['loss_train']

In [ ]:
hyperparameter_log['training_accuracy']

In [ ]:
hyperparameter_log['valid_accuracy']

In [ ]:
d = 3
myFile = hdf5.open('test_pred_embedding_h_'.. d .. '.f5','r')
test_pred_3 = myFile:all()
myFile:close()
max,argmax_3 = test_pred_3['test_pred']:max(2)

In [ ]:
test_pred_3['test_pred']:size()

In [ ]:
filename = 'tocsv_3.f5'
myFile = hdf5.open(filename, 'w')
myFile:write('test_pred', argmax_3)
myFile:close()  

In [ ]:
d = 4
myFile = hdf5.open('test_pred_embedding_h_'.. d .. '.f5','r')
test_pred_4 = myFile:all()
myFile:close()
max,argmax_4 = test_pred_4['test_pred']:max(2)

In [ ]:
filename = 'tocsv_4.f5'
myFile = hdf5.open(filename, 'w')
myFile:write('test_pred', argmax_4)
myFile:close() 

## Fixing the embedings:

In [ ]:
-- Loading the data

-- dimension
nwords = data['nwords'][1]
ncap = 4
nclasses = data['nclasses'][1]

-- Training data
train_output = data['train_output']
train_new = torch.cat(data['train_input_word_windows'],
    torch.add(data['train_input_cap_windows'], 100002),2)

-- Validation data
valid_output = data['valid_output']
valid_new = torch.cat(data['valid_input_word_windows'],
    torch.add(data['valid_input_cap_windows'],100002) ,2)

-- Test data
test_new = torch.cat(data['test_input_word_windows'],
    torch.add(data['test_input_cap_windows'],100002) ,2)

-- Formating the dataset to train
dataset={};
for i=1,train_new:size(1) do 
  dataset[i] = {train_new[i]:view(1,10), train_output[i]}
end
function dataset:size() return train_new:size(1) end -- 100 examples

In [4]:
dim_hidden2_list2 = torch.DoubleTensor({50, 60, 80})
num_parameters2 = dim_hidden2_list2:size(1)

In [ ]:
num_parameters2

In [10]:
-- Metric storage

-- Initialization
loss_train2 = torch.zeros(num_parameters2, 20) -- Store every 1 epochs
training_accuracy2 = torch.zeros(num_parameters2, 20) -- Store every 1 epochs
valid_accuracy2 = torch.zeros(num_parameters2, 20) -- Store every 1 epochs
training_time2 = torch.zeros(num_parameters2)
test_pred2 = torch.zeros(test_new:size(1),45)

In [11]:
-- Script

-- REquired value because of the embeddings
dim_hidden = 50

for d = 1,dim_hidden2_list2:size(1) do
--for d=1,1 do
    dim_hidden2 = dim_hidden2_list2[d]

    --Define the model
    neuralnet_wc = build_nn(nwords, ncap, nclasses, dim_hidden, dim_hidden2, data['word_embeddings'])
    criterion = nn.ClassNLLCriterion()
    
    -- Training
    timer = torch.Timer()
    trainer = nn.StochasticGradient(neuralnet_wc, criterion)
    trainer.learningRate = 0.01
    trainer.maxIteration = 1
    
    for j=1, 20 do
        trainer:train(dataset)
        neuralnet_wc:get(1).weight:narrow(1, 1, nwords):copy(data['word_embeddings']:narrow(1, 1, nwords))
        -- Pred on train
        pred_val = neuralnet_wc:forward(valid_new)
        valid_accuracy2[d][j] = compute_accuracy(pred_val, valid_output)
        
        pred_train = neuralnet_wc:forward(train_new)
        training_accuracy2[d][j] = compute_accuracy(pred_train, train_output)
        loss_train2[d][j] = criterion:forward(pred_train, train_output)
    end

    training_time2[d] = timer:time().real
    test_pred2 = neuralnet_wc:forward(test_new)
    
    -- Saving (rewriting the files with update at each iteration)
    filename = 'log_hyper_embedding2.f5'
    myFile = hdf5.open(filename, 'w')
    myFile:write('valid_accuracy2', valid_accuracy2)
    myFile:write('training_accuracy2', training_accuracy2)
    myFile:write('training_time2', training_time2)
    myFile:write('loss_train2', loss_train2)
    myFile:close()
    
    -- Saving the pred for each hyperparameter
    filename = 'test_pred_embedding_h_'.. d .. '_2.f5'
    myFile = hdf5.open(filename, 'w')
    myFile:write('test_pred', test_pred2)
    myFile:close()    
    
end

# StochasticGradient: training	


# current error = 0.2993823805604	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.2993823805604	


# StochasticGradient: training	


# current error = 0.22768876417716	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.22768876417716	


# StochasticGradient: training	


...

# StochasticGradient: training	


# current error = 0.17542851477198	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.17542851477198	


# StochasticGradient: training	


# current error = 0.17400905746581	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.17400905746581	


...rs/virgileaudi/torch/install/share/lua/5.1/hdf5/file.lua:12: HDF5File: fileID -1 is not valid
stack traceback:
	[C]: in function 'error'
	...rs/virgileaudi/torch/install/share/lua/5.1/hdf5/file.lua:12: in function '__init'
	...s/virgileaudi/torch/install/share/lua/5.1/torch/init.lua:91: in function <...s/virgileaudi/torch/install/share/lua/5.1/torch/init.lua:87>
	[C]: in function 'open'
	[string "-- Script..."]:37: in main chunk
	[C]: in function 'xpcall'
	.../virgileaudi/torch/install/share/lua/5.1/itorch/main.lua:179: in function <.../virgileaudi/torch/install/share/lua/5.1/itorch/main.lua:143>
	.../virgileaudi/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...rgileaudi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...rgileaudi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...rgileaudi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	.../virgileaudi/torch/install/share/lua/5.1/itor

In [ ]:
neuralnet_wc:get(1).weight:size()

In [16]:
training_accuracy2

Columns 1 to 10
 0.6333  0.6696  0.5631  0.6084  0.6280  0.6356  0.6716  0.5664  0.6027  0.6449
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000

Columns 11 to 20
 0.6734  0.5784  0.6078  0.5377  0.5950  0.6517  0.6828  0.6016  0.6042  0.5810
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
[torch.DoubleTensor of size 3x20]



### ADD LAYER:

In [24]:
-- Helper

function compute_accuracy(pred, true_)
    max,argmax = pred:max(2)
    acc = 0
    for i = 1, true_:size(1) do
        if argmax[i][1] == true_[i] then
            acc = acc + 1
        end
    end
    score = acc/true_:size(1)
    
    return score
end

-- If words_embeddings is nil, weight are initialized randomly by torch
function build_nn2(nwords, ncap, nclasses, dim_hidden, dim_hidden2, dim_hidden3, word_embeddings)
    --Define the module
    neuralnet_wc = nn.Sequential()

    par = nn.LookupTable(nwords + ncap,dim_hidden)
    
    -- Adding the embeddings
    if word_embeddings then
        par.weight:narrow(1, 1, nwords):copy(word_embeddings:narrow(1, 1, nwords))
    end
    neuralnet_wc:add(par)

    neuralnet_wc:add(nn.View(1,-1,10*dim_hidden))
    neuralnet_wc:add(nn.Squeeze()) -- this layer is to go from a 1xAxB tensor to AxB dimensional tensor (https://groups.google.com/forum/#!topic/torch7/u4OEc0GB74k)
    neuralnet_wc:add(nn.Linear(10*dim_hidden,dim_hidden2))
    neuralnet_wc:add(nn.HardTanh())
    neuralnet_wc:add(nn.Linear(dim_hidden2, dim_hidden3))
    neuralnet_wc:add(nn.HardTanh())
    neuralnet_wc:add(nn.Linear(dim_hidden3, nclasses))
    neuralnet_wc:add(nn.LogSoftMax())
    
    return neuralnet_wc
end

In [22]:
-- Metric storage

-- Initialization
loss_train3 = torch.zeros(2, 5) -- Store every 4 epochs
training_accuracy3 = torch.zeros(2, 5) -- Store every 4 epochs
valid_accuracy3 = torch.zeros(2, 5) -- Store every 4 epochs
training_time3 = torch.zeros(2)
test_pred3 = torch.zeros(test_new:size(1),45)

In [ ]:
-- Script

--Define the model
neuralnet_wc_1 = build_nn2(nwords, ncap, nclasses, 50, 50, 50 , data['word_embeddings'])
criterion = nn.ClassNLLCriterion()
    
-- Training
timer = torch.Timer()
trainer = nn.StochasticGradient(neuralnet_wc_1, criterion)
trainer.learningRate = 0.01
trainer.maxIteration = 4
    
for j=1, 5 do
    trainer:train(dataset)
        
        -- Pred on train
    pred_val = neuralnet_wc_1:forward(valid_new)
    valid_accuracy3[1][j] = compute_accuracy(pred_val, valid_output)
        
    pred_train = neuralnet_wc_1:forward(train_new)
    training_accuracy3[1][j] = compute_accuracy(pred_train, train_output)
    loss_train3[1][j] = criterion:forward(pred_train, train_output)
end

training_time3[1] = timer:time().real
test_pred3_1 = neuralnet_wc_1:forward(test_new)



# StochasticGradient: training	


In [ ]:
-- Script

--Define the model
neuralnet_wc_2 = build_nn2(nwords, ncap, nclasses, 50, 70, 50 , data['word_embeddings'])
criterion = nn.ClassNLLCriterion()
    
-- Training
timer = torch.Timer()
trainer = nn.StochasticGradient(neuralnet_wc_2, criterion)
trainer.learningRate = 0.01
trainer.maxIteration = 4
    
for j=1, 5 do
    trainer:train(dataset)
        
        -- Pred on train
    pred_val = neuralnet_wc_2:forward(valid_new)
    valid_accuracy3[2][j] = compute_accuracy(pred_val, valid_output)
        
    pred_train = neuralnet_wc_2:forward(train_new)
    training_accuracy3[2][j] = compute_accuracy(pred_train, train_output)
    loss_train3[2][j] = criterion:forward(pred_train, train_output)
end

training_time3[2] = timer:time().real
test_pred3_1 = neuralnet_wc_1:forward(test_new)



In [ ]:
-- Saving (rewriting the files with update at each iteration)
filename = 'log_hyper_embedding.f5'
myFile = hdf5.open(filename, 'w')
myFile:write('valid_accuracy', valid_accuracy)
myFile:write('training_accuracy', training_accuracy)
myFile:write('training_time', training_time)
myFile:write('loss_train', loss_train)
myFile:close()

-- Saving the pred for each hyperparameter
filename = 'test_pred_embedding_h_'.. d .. '.f5'
myFile = hdf5.open(filename, 'w')
myFile:write('test_pred', test_pred3)
myFile:close()    